# Netflix Movies & TV Shows — End-to-End Analytics (Python → SQL → Power BI)
This notebook cleans the dataset, creates analysis-ready features, runs EDA, and exports a cleaned CSV used by SQL & Power BI.


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
RAW_PATH = Path('data/raw/netflix_titles.csv')
PROCESSED_PATH = Path('data/processed/netflix_clean.csv')

df = pd.read_csv(RAW_PATH)
df.shape


## 1) Data quality check


In [ ]:
df.isna().mean().sort_values(ascending=False).head(10)


## 2) Cleaning + feature engineering


In [ ]:
clean = df.copy()

for col in ['director','cast','country','rating']:
    clean[col] = clean[col].fillna('Unknown')

clean['date_added'] = pd.to_datetime(clean['date_added'], errors='coerce')
clean['added_year'] = clean['date_added'].dt.year
clean['added_month'] = clean['date_added'].dt.month

# duration -> minutes or seasons

def parse_duration(d):
    if pd.isna(d):
        return (np.nan, np.nan)
    m = re.match(r'(\d+)\s*(min|Season|Seasons)', str(d))
    if not m:
        return (np.nan, np.nan)
    val = int(m.group(1))
    unit = m.group(2)
    return (val, np.nan) if unit == 'min' else (np.nan, val)

mins, seasons = [], []
for d in clean['duration']:
    mi, se = parse_duration(d)
    mins.append(mi); seasons.append(se)

clean['duration_minutes'] = mins
clean['duration_seasons'] = seasons
clean['is_movie'] = (clean['type']=='Movie').astype(int)

clean['genres'] = clean['listed_in'].fillna('Unknown')
clean['primary_genre'] = clean['genres'].str.split(',').str[0].str.strip()
clean['countries_list'] = clean['country'].fillna('Unknown').str.split(',').apply(lambda xs: ','.join([x.strip() for x in xs]) if isinstance(xs,list) else 'Unknown')
clean['genres_list'] = clean['genres'].str.split(',').apply(lambda xs: ','.join([x.strip() for x in xs]) if isinstance(xs,list) else 'Unknown')

clean.to_csv(PROCESSED_PATH, index=False)
clean.head()


## 3) Exploratory analysis


In [ ]:
type_counts = clean['type'].value_counts()
plt.figure(figsize=(6,3.5))
type_counts.plot(kind='bar')
plt.title('Movies vs TV Shows')
plt.ylabel('Titles')
plt.tight_layout()
plt.show()


In [ ]:
# explode for multi-valued fields
country_exploded = clean.assign(country=clean['country'].str.split(',')).explode('country')
country_exploded['country'] = country_exploded['country'].str.strip().fillna('Unknown')

top_countries = country_exploded['country'].value_counts().head(10)
plt.figure(figsize=(7,4))
top_countries.sort_values().plot(kind='barh')
plt.title('Top 10 Countries by # Titles')
plt.tight_layout()
plt.show()


In [ ]:
genre_exploded = clean.assign(genre=clean['listed_in'].str.split(',')).explode('genre')
genre_exploded['genre'] = genre_exploded['genre'].str.strip().fillna('Unknown')

top_genres = genre_exploded['genre'].value_counts().head(10)
plt.figure(figsize=(7,4))
top_genres.sort_values().plot(kind='barh')
plt.title('Top 10 Genres by # Titles')
plt.tight_layout()
plt.show()


In [ ]:
added_by_year = clean.dropna(subset=['added_year']).groupby('added_year').size().sort_index()
plt.figure(figsize=(7,4))
added_by_year.plot(marker='o')
plt.title('Titles Added by Year')
plt.ylabel('Titles')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## 4) Key business takeaways (from this dataset)
- The catalog is **movie-heavy**, but TV shows are a meaningful minority (good for retention via series).
- The **US and India** dominate title counts, suggesting both supply concentration and localization opportunity.
- A small number of genres account for a large share of the library (portfolio concentration).
- 2018–2020 show a strong ramp in titles added (expansion phase), with a visible dip afterward.
